In [1]:
pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.


In [5]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

In [6]:
import warnings 
warnings.filterwarnings('ignore')

In [7]:
import gensim
from gensim import corpora
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

C:\Users\Deepak\anaconda3\lib\site-packages\plotly\express\imshow_utils.py:24: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Deepak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Deepak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
df = pd.read_csv("Resume.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()

,ID,Resume_str,Resume_html,Category
550,13809698,PHLEBOTOMIST Professional Sum...,"<div class=""fontsize fontface vmargins hmargin...",ADVOCATE
2221,19176318,SENIOR FINANCIAL ANALYST Prof...,"<div class=""fontsize fontface vmargins hmargin...",BANKING
2077,36107065,RESEARCH AND STUDY ELABORATION MANAGE...,"<div class=""fontsize fontface vmargins hmargin...",PUBLIC-RELATIONS
1022,27624540,SALES ASSOCIATE/GOLF SALES ASSOCIATE ...,"<div class=""fontsize fontface vmargins hmargin...",SALES
1731,12518008,ENGINEERING MANAGER Profess...,"<div class=""MPR skn-cbg1 fontsize fontface vma...",ENGINEERING


In [9]:
nlp = spacy.load("en_core_web_lg")
skill_pattern_path = "jz_skill_patterns.jsonl"

In [10]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(skill_pattern_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [11]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [12]:
clean = []
for i in range(data.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [13]:
data["Clean_Resume"] = clean
data["skills"] = data["Clean_Resume"].str.lower().apply(get_skills)
data["skills"] = data["skills"].apply(unique_skills)
data.head()

,ID,Resume_str,Resume_html,Category,Clean_Resume,skills
550,13809698,PHLEBOTOMIST Professional Sum...,"<div class=""fontsize fontface vmargins hmargin...",ADVOCATE,phlebotomist professional summary reliable hig...,"[monitoring, mobile, support, testing]"
2221,19176318,SENIOR FINANCIAL ANALYST Prof...,"<div class=""fontsize fontface vmargins hmargin...",BANKING,senior financial analyst professional summary ...,"[monitoring, database, tower, business, market..."
2077,36107065,RESEARCH AND STUDY ELABORATION MANAGE...,"<div class=""fontsize fontface vmargins hmargin...",PUBLIC-RELATIONS,research study elaboration manager public rela...,"[workflow, database, business, release, data a..."
1022,27624540,SALES ASSOCIATE/GOLF SALES ASSOCIATE ...,"<div class=""fontsize fontface vmargins hmargin...",SALES,sale associate golf sale associate executive s...,"[material, support]"
1731,12518008,ENGINEERING MANAGER Profess...,"<div class=""MPR skn-cbg1 fontsize fontface vma...",ENGINEERING,engineering manager professional summary exper...,"[mechanical engineering, design, support, engi..."


In [2]:
import fitz
with fitz.open('test_resume.pdf') as doc:  # open document
    text = chr(12).join([page.get_text() for page in doc])
print(text)

DEEPAK S
deepaksridar2@gmail.com ⋄ www.linkedin.com/in/deepak035 ⋄ https://github.com/Its-Deepak-S
OBJECTIVE
I am a budding ML Engineer who’s passionate about Deep Learning and Data Science. I am always looking for
opportunities to utilize my skills and knowledge on real - world tasks and upskilling myself in the process.
EDUCATION
B.Tech CSE (Specialization in AIML), Jain(Deemed-to-be University)
Expected 2025
Class 12, Chavara Public School
2020-2021
SKILLS
Technical Skills
Deep Learning, Machine Learning, Python, Computer Vision, MySQL, Java, JavaScript
Soft Skills
Communication, Problem Solving, Teamwork, Interpersonal Skills
Languages
English, Malayalam, Hindi
EXPERIENCE
Intern in Artificial Intelligence
05/23 - 06/23
Corizo
Intern in Full-stack Development
06/23 - 07/23
Capabl
PROJECTS
Image Generation GAN: Image Generation GAN leverages Generative Adversarial Networks (GANs) on Fashion
MNIST data to produce realistic fashion images, showcasing the power of deep learning in gener

In [3]:
input_resume = text

In [14]:
colors = {
    "Job-Category": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
    "SKILL": "linear-gradient(90deg, #9BE15D, #00E3AE)",
    "ORG": "#ffd966",
    "PERSON": "#e06666",
    "GPE": "#9fc5e8",
    "DATE": "#c27ba0",
    "ORDINAL": "#674ea7",
    "PRODUCT": "#f9cb9c",
}
options = {
    "ents": [
        "Job-Category",
        "SKILL",
        "ORG",
        "PERSON",
        "GPE",
        "DATE",
        "ORDINAL",
        "PRODUCT",
    ],
    "colors": colors,
}
sent = nlp(data["Resume_str"].iloc[5])
displacy.render(sent, style="ent", jupyter=True, options=options)

In [15]:
sent2 = nlp(input_resume)
displacy.render(sent2, style="ent", jupyter=True, options=options)

In [22]:
inp_skills="""Computer Vision, Machine Learning, MYSQL, JavaScript"""

In [23]:
req_skills = inp_skills.lower().split(",")
resume_skills = unique_skills(get_skills(input_resume.lower()))
score = 0
for x in req_skills:
    if x in resume_skills:
        score += 1
req_skills_len = len(req_skills)
match = round(score / req_skills_len * 100, 1)

print(f"The current Resume is {match}% matched to your requirements")

The current Resume is 25.0% matched to your requirements
